In [19]:
import pandas as pd
import numpy as np

def load_coords_cache(cache_file: str) -> dict:
    '''
    Charge les coordonnées depuis le fichier de cache avec mise en cache Streamlit
    '''
    try:
        # Lire le fichier CSV
        df = pd.read_csv(cache_file, encoding='utf8',sep=';')
        # Creer le dictionnaire {code postale : {lat,lon}}
        coords_dict = {}
        for _, row in df.iterrows():
            coords_dict[row['code_postal']] = (row['lon'], row['lat'])
        return coords_dict
    except Exception as e:
        print(f"Erreur lors du chargement du cache {cache_file} : {str(e)}")
        if 'dept' in cache_file.lower():
            return DEFAULT_DEPT_COORDS
        return {}

a = load_coords_cache('villes_france_V1.csv')

print(a)
print(a.get('78300'))  # Exemple d'accès aux coordonnées du code postal 78300

{'1190': (46.4667, 4.96667), '1290': (46.2167, 4.93333), '1130': (46.21, 5.74139), '1250': (46.1333, 5.4), '1230': (45.95, 5.53333), '1620': (46.2833, 4.9), '1110': (46.0, 5.5), '1630': (46.2167, 5.95), '1990': (46.1, 4.85), '1660': (46.2333, 5.03333), '1150': (45.9167, 5.3), '1430': (46.0333, 5.53333), '1800': (45.9206, 5.22223), '1350': (45.8167, 5.71667), '1600': (45.9667, 4.83333), '1510': (45.8333, 5.61667), '1160': (46.0333, 5.31667), '1460': (46.1728, 5.55251), '1400': (46.0667, 4.98333), '1260': (45.95, 5.65), '1500': (46.0, 5.35), '1680': (45.75, 5.53333), '1560': (46.4667, 5.1), '1851': (46.3333, 5.25), '1340': (46.3333, 5.18333), '1370': (46.3333, 5.38333), '1300': (45.75, 5.76667), '1450': (46.1, 5.48333), '1320': (45.9884, 5.26862), '1310': (46.2, 5.13333), '1140': (46.1167, 4.86667), '1270': (46.3667, 5.35), '1330': (45.9833, 4.96667), '1390': (45.8667, 4.96667), '1360': (45.7667, 5.2), '1960': (46.1833, 5.2), '1480': (46.0484, 4.76473), '1120': (45.85, 5.06667), '1640': 

In [35]:
def load_data() -> pd.DataFrame:
    """
    Charge les données depuis l'URL avec mise en cache Streamlit
    """
        
    try:
        df = pd.read_csv('liste-ps-20260105-023058.csv', encoding='utf-8', sep=';', # Spécifie l'encodage
            on_bad_lines='skip')  # Saute les lignes mal formatées
           
        return df
        
    except Exception as e:
        print(f"Erreur lors du chargement des données : {str(e)}")
        return pd.DataFrame()


In [ ]:
def preprocess_data(data: pd.DataFrame, ville_cache: dict) -> pd.DataFrame:
    """
    Prétraite les données avec mise en cache Streamlit
    """
    if data.empty:
        #print("Les données sont vides, aucun prétraitement effectué.")
        return data
    
    # Ajouter les coordonnées des villes (seulement si la colonne existe)
    print(data.columns)
    is_coordonnees_code_postal = 'coordonnees_code_postal' in data.columns
    print(f"Colonne 'coordonnees_code_postal' présente : {is_coordonnees_code_postal}")
    if is_coordonnees_code_postal:
        def get_coordinates(code_postal):
            if pd.isna(code_postal):
                return None, None
            
            key = str(int(code_postal))
            
            # Cherche dans le cache
            coords = ville_cache.get(key, (None, None))
            return coords[0], coords[1]  # (latitude, longitude)
        coordinates = data['coordonnees_code_postal'].apply(get_coordinates)
        data['Latitude_Ville'] = coordinates.apply(lambda x: x[0])
        data['Longitude_Ville'] = coordinates.apply(lambda x: x[1])
      
        non_null = data['Latitude_Ville'].notna().sum()
        print(f"✅ {non_null}/{len(data)} coordonnées trouvées")
    else:
        # Si pas de code postal ville, initialiser les colonnes avec des valeurs nulles
        #print("Aucun code postal de ville trouvé, initialisation des colonnes de coordonnées à None.")
        data['Latitude_Ville'] = None
        data['Longitude_Ville'] = None
    
    
    return data 

In [39]:
df = load_data()

df = preprocess_data(df, a)

print(df.head())

Index(['ps_activite_nom', 'ps_activite_prenom', 'ps_activite_civilite',
       'ps_activite_raison_sociale', 'ps_activite_carte_vitale',
       'ps_activite_apcv', 'specialite_code', 'specialite_libelle',
       'activite_particuliere_code', 'activite_particuliere_libelle',
       'type_ps_code', 'type_ps_libelle', 'coordonnees_num_tel',
       'coordonnees_voie', 'coordonnees_complement', 'coordonnees_lieu_dit',
       'coordonnees_code_postal', 'coordonnees_ville', 'option_tarifaire_code',
       'option_tarifaire_libelle', 'secteur_conventionnel_code',
       'secteur_conventionnel_libelle', 'nature_exercice_code',
       'nature_exercice_libelle'],
      dtype='object')
Colonne 'coordonnees_code_postal' présente : True
✅ 417869/545265 coordonnées trouvées
  ps_activite_nom ps_activite_prenom ps_activite_civilite  \
0          MAYAUD            NORBERT                    M   
1       DOPSOUMNA            OUMAROU                    M   
2        ORSZULAK             RUDOLF           